In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import cv2
import numpy as np
import math
from scipy.stats.stats import pearsonr
from skimage.metrics import peak_signal_noise_ratio

import csv

In [ ]:
def append_list_as_row(csv_file, f_num, psnr_num):
    with open(csv_file, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerows([f_num])
        writer.writerows([psnr_num])


In [ ]:
def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 255]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(255.0 / math.sqrt(mse))

In [ ]:
def normalize(y_true, y_pred):
    # cast the target images to integer
    y_true = y_true * 255.0
    y_true = y_true.astype(np.uint8)
    y_true = np.clip(y_true, 0, 255)
    
    # cast the predicted images to integer
    y_pred = y_pred * 255.0
    y_pred = y_pred.astype(np.uint8)
    y_pred = np.clip(y_pred, 0, 255)
    # return the psnr
    return y_true, y_pred

In [ ]:
PSNR = []
Pearson_CC = []
frames = []
for f in range(1,513):
    print('f :', f)
    frames.append(f)
    os.chdir('/aijjeh_odroid_sensors/aidd/data/interim/exp/L3_S4_B/compressed/389286p_512x512/')
    ref_data = cv2.imread('frame_%d_389286p_512x512.png' %f, 0)        
    ref_img = np.asarray(ref_data) # / 255.0
    
    
    os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Experimental/Uniform_mesh_2nd_model/Upated_version_FFT2D/')
    pred_data = cv2.imread('Ijjeh_SR_Pred_output_%d_frame_UNIFORM_MESH_custom_loss.png' %f, 0) 
    pred_img = np.asarray(pred_data) # / 255.0        
    
    # ref_img = ref_img[113:213, 256-50:256+50]
    # pred_img = pred_img[113:213, 256-50:256+50]
    
    ref_img = ref_img - np.mean(ref_img)
    pred_img = pred_img - np.mean(pred_img)
    
    temp_pearson = np.corrcoef(ref_img.flat, pred_img.flat)
    print('Pearson_CC :', temp_pearson[0][1])
    Pearson_CC.append(temp_pearson[0][1])
    
    # ref_img, pred_img = normalize(ref_img, pred_img)
    
    
    temp_psnr = peak_signal_noise_ratio(ref_img, pred_img, data_range=128)
    print('PSNR :', temp_psnr)
    PSNR.append(temp_psnr)
    
    # damage_area = np.zeros((512,512))
    # damage_area[113:213, 256-50:256+50]= 255
    # ref_img = ref_img+ damage_area
    # fig = plt.figure(figsize=(20, 20))  
    # damage_area = ref_img
    # ax = fig.add_subplot(111)
    # ax.imshow(ref_img)  
    # ax.add_patch(patches.Rectangle((256-50, 112), 100, 100, linewidth=1, edgecolor='r', facecolor='none')) 
    # ax.axis('off')
    # plt.show()
    
os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/')    
file_pearson = 'Pearson_CC_exp_512_frames__custim_loss_spatial_Fourier.csv'
file_psnr = 'PSNR_exp_512_frames_custim_loss_spatial_Fourier.csv'
append_list_as_row(file_pearson, frames, Pearson_CC)
append_list_as_row(file_psnr, frames, PSNR)
# 

In [ ]:
PSNR = np.asarray(PSNR)
Pearson_CC = np.asarray(Pearson_CC)
print(np.mean(Pearson_CC))
print(np.mean(PSNR))

In [ ]:
print((np.min(Pearson_CC)))

In [ ]:
print(np.max(PSNR))

In [ ]:
os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Experimental/Uniform_mesh_2nd_model/Compression_ration_0.215/')
plt.figure(figsize=(20/2.53,10/2.53), dpi=600)
plt.plot(Pearson_CC)
plt.xlabel('frame')
plt.ylabel('Pearson CC')
# plt.savefig("Pearson_CC_for_512_frames_exp_delamination_region.png")
plt.show()

In [ ]:
os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Experimental/Uniform_mesh_2nd_model/Compression_ration_0.215/')
plt.figure(figsize=(20/2.53,10/2.53), dpi=600)
plt.plot(PSNR)
plt.xlabel('frame')
plt.ylabel('PSNR')
# plt.savefig("PSNR_for_512_frames_exp_delamination_region.png")
plt.show()